In [1]:
import inspect
import numpy as np
import numdifftools as nd
from scipy import optimize

https://stackoverflow.com/questions/78145811/python-scipy-optimize-minimize-trust-constr-and-hessian-output/78146124#78146124

In [2]:
def model(x, a, b):
    return a * x + b

In [3]:
def loss_factory(func, x, y, s):
    ddof = len(inspect.signature(func).parameters) - 1
    def wrapped(p):
        return np.sum(np.power((y - func(x, *p)) / s, 2)) / (x.size - ddof)
    return wrapped

In [4]:
np.random.seed(123456)
x = np.linspace(-1, 1, 30)
s = 0.1 * np.ones_like(x)
y = model(x, 2, 3)
n = s * np.random.normal(size=x.size)
y += n

In [5]:
loss = loss_factory(model, x, y, s)

In [6]:
s

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1])

In [7]:
sol = optimize.minimize(loss, x0=[1., 1.], method="trust-constr")
sol

         cg_niter: 22
     cg_stop_cond: 4
           constr: []
      constr_nfev: []
      constr_nhev: []
      constr_njev: []
   constr_penalty: 1.0
 constr_violation: 0
   execution_time: 0.1094207763671875
              fun: 0.7769476219857044
             grad: array([2.24757227e-08, 9.80527794e-08])
              jac: []
  lagrangian_grad: array([2.24757227e-08, 9.80527794e-08])
          message: '`xtol` termination condition is satisfied.'
           method: 'equality_constrained_sqp'
             nfev: 39
             nhev: 0
              nit: 19
            niter: 19
             njev: 13
       optimality: 9.805277940760494e-08
           status: 2
          success: True
        tr_radius: 7.604991998619307e-09
                v: []
                x: array([1.98896756, 2.96343096])

In [8]:
H = nd.Hessian(loss)(sol.x)
H

array([[ 7.63546798e+01, -1.22821246e-14],
       [-1.22821246e-14,  2.14285714e+02]])

In [9]:
C = np.linalg.inv(H)
C

array([[1.30967742e-02, 7.50662328e-19],
       [7.50662328e-19, 4.66666667e-03]])

In [10]:
sy = np.sqrt(np.diag(C))
sy

array([0.11444114, 0.06831301])